In [11]:
#Downloading and formatting the dataset

from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
symbols_list = sp500['Symbol'].unique().tolist()

end_date = dt.datetime.now()
start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

df = yf.download(tickers = symbols_list, start = start_date, end = end_date)
df = df.stack()
df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()

[*********************100%***********************]  503 of 503 completed


In [56]:
#Computing the technical indicators

df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - (2 * np.log(2) - 1) * ((np.log(df['adj close']) - np.log(df['open'])) ** 2)
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length = 20))

df['bb_low'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,0])
df['bb_mid'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,1]) 
df['bb_high'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level = 1, group_keys = False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20)
    if macd is not None:
        return macd.iloc[:, 0].sub(macd.iloc[:, 0].mean()).div(macd.iloc[:, 0].std())
    else:
        return pd.Series([np.nan] * len(close), index=close.index)

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close'] * df['volume'])/1e6

In [71]:
#Aggregate to monthly level and filter top 150 most liquid stocks for each month
'''This is done to reduce training time for the ML model and experiment with strats'''
#This computes the average monthly dollar volume to give a sense of liquidity

last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume',
                                                          'open','high','low','close']]

data = pd.concat([df.unstack('ticker')['dollar_volume'].resample('M').mean().stack().to_frame('dollar_volume'),
          df.unstack()[last_cols].resample('M').last().stack('ticker')], axis = 1).dropna()

In [72]:
#Calculating the 5 year rolling average rolling average for each stock
#and use this value to filter out top 150 most liquid stocks for each month

data['dollar_volume'] = data['dollar_volume'].unstack('ticker').rolling(5*12).mean().stack()

data['dollar_vol_rank'] = data.groupby('date')['dollar_volume'].rank(ascending = False)

#We are finding the 150 most liquid stocks then dropping the columns that we dont need
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis = 1)

In [73]:
#Calculating monthly returns for different time horizons as features
'''To capture time series dynamics that reflect, for instance, momentum patterns, we
compute historical returns using the method .pct_change(lag), that is, returns over
various monthly periods as identified by lags'''

data

adj close  garman_klass_vol        rsi  \
date                      ticker                                            
2021-11-30 00:00:00+00:00 AAPL    162.814850          0.000474  68.678424   
                          ABBV    102.887581         -0.006007  52.881765   
                          ABT     118.895699         -0.001824  50.504816   
                          ACN     341.584473         -0.001030  51.087318   
                          ADBE    669.849976          0.000536  55.431004   
...                                      ...               ...        ...   
2024-11-30 00:00:00+00:00 VZ       41.360001          0.000187  43.707585   
                          WFC      64.680000          0.000082  64.651722   
                          WMT      82.190002          0.000066  58.968978   
                          XOM     114.949997          0.000253  41.476972   
                          ZTS     181.949997          0.000187  43.479901   

                                    bb_low    bb_mid   bb_high       atr  \
date                      ticker                                           
2021-11-30 00:00:00+00:00 AAPL    4.962430  5.030926  5.099421  0.581527   
                          ABBV    4.641545  4.656319  4.671094 -0.639553   
                          ABT     4.771433  4.797957  4.824481  0.778265   
                          ACN     5.832630  5.858994  5.885359  0.710208   
                          ADBE    6.464803  6.501262  6.537721  1.521722   
...                                    ...       ...       ...       ...   
2024-11-30 00:00:00+00:00 VZ      3.731670  3.776617  3.821565 -0.019533   
                          WFC     4.061677  4.156092  4.250506  0.553108   
                          WMT     4.382265  4.410219  4.438172  1.114684   
                          XOM     4.755817  4.798220  4.840622  0.695394   
                          ZTS     5.185172  5.235566  5.285961  0.502353   

                                      macd  
date                      ticker            
2021-11-30 00:00:00+00:00 AAPL    1.322963  
                          ABBV    0.361369  
                          ABT     0.095065  
                          ACN     0.416085  
                          ADBE    0.961097  
...                                    ...  
2024-11-30 00:00:00+00:00 VZ     -1.002509  
                          WFC     2.345227  
                          WMT     1.047007  
                          XOM    -0.567920  
                          ZTS    -1.245712  

[5513 rows x 8 columns]